In [12]:
import cv2
import imutils
import pytesseract

In [13]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\\tesseract'

In [23]:
img = cv2.imread('q.jpg')

# Width, Height. Converting to 4:3 ratio
# img = cv2.resize(img, (300, 209))

img = imutils.resize(img, width=300)

cv2.imshow("original image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

img.shape

(84, 300, 3)

In [24]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow("Gray image", img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
# It is used for removing noise without effecting edge information.
# 2nd parameter indicated the diameter taken to consider the neighbours.
# 3rd paramter(Sigma color) specifies farther colors in the neighbourhood will be mixed if it is high 
# 4th parameter(Sigma space) specifies how much the distant pixels has to be considered
smooth_img_gray = cv2.bilateralFilter(img_gray, 11, 17, 17) 
cv2.imshow("smoothened image", img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

# We can't use gaussian because it smoothnes edges as well

# gray_image = cv2.GaussianBlur(gray_image, (3, 3), 0)

# cv2.imshow("Smooth image", gray_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [26]:
# ret, smooth_img_gray = cv2.threshold(smooth_img_gray, 127, 255, cv2.THRESH_BINARY)
# cv2.imshow("smoothened image", img_gray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [27]:
# Canny Edge detection is an Algorithm used to detect the edges in an image and it consists of 4 major steps:
# Reduce Noise using Gaussian Smoothing.
# Compute image gradient using Sobel filter.
# Doing thresholding
# Apply Non-Max Suppression
# Finally, apply Hysteresis thresholding which that 2 threshold values T_upper and T_lower which is used in the Canny() function.

# First argument is our input image. Second and third arguments are our minVal and maxVal respectively. 

edges = cv2.Canny(smooth_img_gray, 30, 200) 
cv2.imshow("edged image", edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
# Contours are defined as the line joining all the points along the boundary of an image that are having the same intensity.
# Using contours we can find the size of the object of interest, and object detection.

# There are three arguments in cv.findContours() function, 
# first one is source image, 
# second is contour retrieval mode, 
# third is contour approximation method.

# And it outputs the contours and hierarchy. Contours is a Python list of all the contours in the image. 
# Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.

contours, new = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
img1 = img.copy()
cv2.drawContours(img1, contours, -1, (0, 255, 0), 3)
cv2.imshow("contours", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
# sorting contours based on the minimum area 30 and ignoring the ones below that.

contours_selected = sorted(contours, key=cv2.contourArea, reverse=True) [:30]
screenCnt = None
img2 = img.copy()
cv2.drawContours(img2, contours_selected, -1, (0, 255, 0), 3)
cv2.imshow("Top 30 contours", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imwrite("contor1.jpg", image2)

In [29]:
# To find the best contour of our expected number plate.

i=0

for cont in contours_selected:
    perimeter = cv2.arcLength(cont, True)
    approx = cv2.approxPolyDP(cont, 0.018 * perimeter, True)
    
    # chooses the contours with four sides as this will probably be our number plate.
    if len(approx) == 4: 
        screenCnt = approx
        x, y, w, h = cv2.boundingRect(cont) 
        new_img=img[y:y+h, x:x+w]
        # cv2.imshow("original image", new_img)
        
        # Stores the new image of the cropped number plate.
        cv2.imwrite('./cropped.png',new_img)
        i+=1
        break

print(screenCnt)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-r2ue8w6k\opencv\modules\imgcodecs\src\loadsave.cpp:753: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [30]:
# This draws the contour selected to be the number plate on our original image.

Cropped_loc = './q.jpg'
cv2.imshow("cropped", cv2.imread(Cropped_loc))
plate = pytesseract.image_to_string(Cropped_loc, lang='eng')
print(plate)
# We are passing the image of the cropped part of the license plate. 
# We are then calling on pytesseract to extract the text on the image.
cleaned_plate = ''.join(c for c in plate if(c.isalnum()))
print("Number plate is:", cleaned_plate)
cv2.waitKey(0)
cv2.destroyAllWindows()

a

TN-41-AJ-4800

Number plate is: aTN41AJ4800


In [ ]:
# If backgroung is mixing with foreground. It is not detecting contour. So, there should be a differentiation from number plate and car
# If image is too far and illumination is not proper contours are not detecting

In [40]:
import cv2
import imutils
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

img = cv2.imread('single.jpg',cv2.IMREAD_COLOR)
img = cv2.resize(img, (600,400) )

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
gray = cv2.bilateralFilter(gray, 13, 15, 15) 

edged = cv2.Canny(gray, 30, 200) 
contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:30]

cn = cv2.drawContours(img, contours, -1, (0, 255, 0), 3)

cv2.imshow("Top 30 contours", cn)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [41]:
screenCnt = None

for c in contours:
    
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
 
    if len(approx) == 4:
        screenCnt = approx
        break

if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
     detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv2.bitwise_and(img,img,mask=mask)

(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

text = pytesseract.image_to_string(Cropped, config='--psm 11')
print("programming_fever's License Plate Recognition\n")
print("Detected license plate Number is:",text)
img = cv2.resize(img,(500,300))
Cropped = cv2.resize(Cropped,(400,200))
cv2.imshow('car',img)
cv2.imshow('Cropped',Cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()

No contour detected


error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-r2ue8w6k\opencv\modules\imgproc\src\drawing.cpp:2598: error: (-215:Assertion failed) reader.ptr != NULL in function 'cvDrawContours'
